In [42]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time
import numpy as np
import argparse, pprint
import matplotlib

from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

# if homeDir not in sys.path:
#     sys.path.append(homeDir)

from utils.playground_utils import *
from src.UVMappingPipeline import UVMappingPipeline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
%reload_ext autoreload
plt.close('all')
pipe = UVMappingPipeline()
p = interactive(pipe.pipeline,
    _img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ),
    line_method = {'Standard Hough Transform':0,'Probabilistic Hough Transform':1},
    input_method = {'Classic Thresholded':0,'Morphed Thresholded':1,'Canny Edges':2},
    e1 = (1,255,1), e2 = (1,255,1),
    ang = (0,360,1), rho = (1,1000,1),
    minLineLength = (0,255,1), maxLineGap = (0,255,1),
    houghThresh = (0,255,1), greyThresh = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = True, use_umap = True, flip_thresh_bin = False
)


row1 = HBox([p.children[0]])
row2 = HBox([p.children[1], p.children[2]], layout = Layout(flex_flow='row wrap'))
row3 = HBox([p.children[3], p.children[4]], layout = Layout(flex_flow='row wrap'))
row4 = HBox([p.children[5], p.children[6], p.children[7], p.children[8]], layout = Layout(display='flex',flex_flow='row'))
row5 = HBox([p.children[9], p.children[10], p.children[11]], layout = Layout(flex_flow='row wrap'))
row6 = HBox([p.children[12], p.children[13], p.children[14]], layout = Layout(flex_flow='row wrap'))
controls = VBox([row1,row2,row3,row4,row5,row6], layout = Layout(display='flex'))
output = p.children[-1]
display(VBox([controls, output]))

VkJveChjaGlsZHJlbj0oVkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oU2VsZWN0KGRlc2NyaXB0aW9uPXUnRmlsZTonLCBsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnMTAwJScpLCBvcHRpb27igKY=


In [41]:
plot_image(pipe.greyV,0)


0

In [ ]:
found_imgs = grab_dir_images(homeDir)

e1 = 2; e2 = 5; houghThresh = 50; greyThresh = 11; cnt_thresh = 500.0
# img = cv2.imread('/home/hunter/devel/vision_playground/test/frames/frame_545.jpg',cv2.IMREAD_GRAYSCALE)
img = cv2.imread('/home/hunter/devel/vision_playground/test/frames/frame_300.jpg',cv2.IMREAD_GRAYSCALE)
umap,vmap, overlay = uvMapping(img)

tmpIn = np.copy(vmap)
tmp = cv2.cvtColor(tmpIn,cv2.COLOR_GRAY2BGR)
n,m,_ = tmp.shape
filtered = np.zeros((n,m,3),dtype=np.uint8)
masking = cv2.THRESH_BINARY
            
kernel = np.ones((2,5),np.uint8)
kernel2 = np.ones((4,4),np.uint8)

ret, grey = cv2.threshold(tmpIn,greyThresh,255,masking)
dilation = cv2.dilate(grey,kernel,iterations = 1)
blur = cv2.GaussianBlur(dilation,(5,5),0)
closing = cv2.morphologyEx(grey,cv2.MORPH_CLOSE,kernel, iterations = 2)

plot_image(grey,3)

# helper_imgs = [tmp,grey,dilation,blur,closing,canny]

In [ ]:
x_subset = np.array(x_subset)
y_subset = np.array(y_subset)

# print(x_subset.shape)
pts = np.array([y_subset,x_subset]).transpose()

plt.figure(2)
plt.clf()
plt.title('Histogram of the image')
plt.plot(x_subset[:], -y_subset[:])
plt.axis('equal')


In [ ]:
cpy = cv2.cvtColor(tmpIn,cv2.COLOR_GRAY2BGR)
mask = cv2.cvtColor(black,cv2.COLOR_BGR2GRAY)
mask_inv = cv2.bitwise_not(mask)

res_hsv = cv2.bitwise_and(cpy, cpy, mask = mask_inv)
plt.figure(3)
plt.clf()
plt.imshow(res_hsv)
plt.show()


masking = cv2.THRESH_BINARY
            
kernel = np.ones((5,2),np.uint8)

ret, grey = cv2.threshold(res_hsv,greyThresh,255,masking)
dilation = cv2.dilate(grey,kernel,iterations = 1)
close = cv2.morphologyEx(grey, cv2.MORPH_CLOSE, (4,4))
blur = cv2.GaussianBlur(dilation,(5,5),0)
plot_image(close,5)

In [ ]:
nonzero = res_hsv.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])

ploty = np.linspace(0, w)


# poly_left = np.poly1d(np.polyfit(
#     left_line_y,
#     left_line_x,
#     deg=1
# ))
# left_x_start = int(poly_left(max_y))
# left_x_end = int(poly_left(min_y))

# line_image = draw_lines(
#     image,
#     [left_x_start, max_y, left_x_end, min_y],
#     thickness=5,
# )
# plt.figure()
# plt.imshow(line_image)
# plt.show()


# Try fitting polynomial to left side
try:
    # Fit a second order polynomial to each
    fit = np.poly1d(np.polyfit(nonzeroy, nonzerox, 3))
    print(fit)
    # Generate x and y values for plotting
    plot_x = fit(ploty)
except:
    print("ERROR: Function 'polyfit' failed for LEFT SIDE!")
    fit = [0, 0]
    plot_x = [0, 0]
    pass

# cv2.line(res_hsv,(int(plot_x[0]),int(ploty[0])),(int(plot_x[-1]),int(ploty[-1])),(0,0,255))
plt.imshow(res_hsv)
plt.plot(ploty,plot_x,'r-')
plt.show()

In [ ]:
print("""
Good Configuration:    for image = test/test_disparity.png)
---------------------------------------------------------------
    
    * Mapping Space       : U-Map
    * Line Finding Method : Probablistic Hough Transform
    * Filtering Used      : Composite Filtering -> Blurring
    * Kernel Size         : (2, 3)
    * Rho, Angle (deg)    :  1, 90
    * Min Line Length     :  11
    * Max Line Gap        :  22
    * Grey Thresholding   :  11
    * Hough Threshold     :  46
    * Contour Threshold   :  500
    
    * Raw Contour Areas        : [2762.0, 1751.0, 4845.5, 2999.0, 7708.5, 1574.0, 398.0])
    * Filtered Contour Areas   : [2762.0, 1751.0, 4845.5, 2999.0, 7708.5, 1574.0])
""")

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression,RANSACRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import (
    LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# ransac_img = cv2.imread("/home/hunter/devel/vision_playground/input-lines.png")
# nonzero_pxls = ransac_img.nonzero()
# nonzero_y = np.array(nonzero_pxls[0]).reshape(-1,1)
# nonzero_x = np.array(nonzero_pxls[1]).reshape(-1,1)


In [ ]:
from sklearn.linear_model import (
    LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

nonzero_y = np.array(nonzero_pxls[0]).reshape(-1,1)
nonzero_x = np.array(nonzero_pxls[1]).reshape(-1,1)
ransac = RANSACRegressor()

estimators = [('OLS', LinearRegression()),
              ('Theil-Sen', TheilSenRegressor(random_state=42)),
              ('RANSAC', RANSACRegressor(random_state=42)),
              ('HuberRegressor', HuberRegressor())]
colors = {'OLS': 'turquoise', 'Theil-Sen': 'gold', 'RANSAC': 'lightgreen', 'HuberRegressor': 'black'}
linestyle = {'OLS': '-', 'Theil-Sen': '-.', 'RANSAC': '--', 'HuberRegressor': '--'}
lw = 3

x_plot = np.linspace(X.min(), X.max())
for title, this_X, this_y in [
        ('Modeling Errors Only', X, y),
        ('Corrupt X, Small Deviants', X_errors, y),
        ('Corrupt y, Small Deviants', X, y_errors),
        ('Corrupt X, Large Deviants', X_errors_large, y),
        ('Corrupt y, Large Deviants', X, y_errors_large)]:
    plt.figure(figsize=(5, 4))
    plt.plot(this_X[:, 0], this_y, 'b+')

    for name, estimator in estimators:
        model = make_pipeline(PolynomialFeatures(3), estimator)
        model.fit(this_X, this_y)
        mse = mean_squared_error(model.predict(X_test), y_test)
        y_plot = model.predict(x_plot[:, np.newaxis])
        plt.plot(x_plot, y_plot, color=colors[name], linestyle=linestyle[name],
                 linewidth=lw, label='%s: error = %.3f' % (name, mse))

    legend_title = 'Error of Mean\nAbsolute Deviation\nto Non-corrupt Data'
    legend = plt.legend(loc='upper right', frameon=False, title=legend_title,
                        prop=dict(size='x-small'))
    plt.xlim(-4, 10.2)
    plt.ylim(-2, 10.2)
    plt.title(title)
plt.show()